# Kahoot Ranking

Este Notebook filtra os vencedores do Kahoot, retornando os primeiros n colocados.

## ▶ Pontuação
### 1. Pontos de pódio
Para cada Kahoot:

🥇1 lugar : 3 pontos

🥈2 lugar : 2 pontos

🥉3 lugar : 1 ponto

### 2. Pontos do Kahoot
O desempate é feito pela pontuação acumulada dos Kahoots. 

In [26]:
import os
import pandas as pd
from unidecode import unidecode
from fuzzywuzzy import process
import re

import logging

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

def clean_string(input_string):
    # Use a regular expression to remove spaces and numbers
    cleaned_string = re.sub(r'[\s\d\W]', '', input_string)

    # Remove accents using unidecode
    string = unidecode(cleaned_string).lower()

    return string

def normalize_name(name, known_names, mapping, threshold=30):
    """
    Normalize a name by checking against a mapping and known names.

    Parameters:
    name (str): The name to normalize.
    known_names (list): A list of known names to compare against.
    mapping (dict): A dictionary mapping names to their normalized forms.
    threshold (int): The minimum score for a match to be considered valid (default is 30).

    Returns:
    str: The normalized name if a match is found; otherwise, returns 'johann'.
    """
    if name in mapping:
        return mapping[name]

    match = process.extractOne(name, known_names)

    if match and match[1] >= threshold:
        return match[0]

    return 'johann'

## Set folder path

In [27]:
# Set folder path
folder_path = "2025-2_ADORACAO"

## Name exception dict 
Define name aliases to substitute (check which names have more than 1 alias)

'alias': 'originalname'

In [38]:
name_alias = {
    'mar+bi=marbi': 'mardabi',
    'marsembi': 'mardabi',
    'marbinoso': 'mardabi',
    'marcombi': 'mardabi',
    'marbinado': 'mardabi',
    'marbix': 'mardabi',
    'mardasuperbi': 'mardabi',
    'bine': 'bidomar',
    'binedomar': 'bidomar',
    'sabrine': 'bidomar',
    'mardomar': 'johann',
    'quadra': 'johann',
    'quadrado': 'johann',
    'bigxand': 'xandao',
    'shaquilleomeal': 'johann',
    'bobberkurwa': 'johann',
    'paracetamal': 'johann',
    'quadroh': 'johann',
    'vaixco ': 'johann',
    'bagriel': 'johann',
    'mardomar': 'johann',
    'quadrado': 'johann',
    'luaraa': 'luara',
    'luaraara': 'luara',
    'kakerlake': 'johann',
    'fmr': 'johann',
    'pirarucu': 'johann',
    'tucunare': 'johann',
    'bambu': 'johann',
    'yej!b': 'yejin',
    'yej!n': 'yejin',
    'tirisco': 'johann',
    'luu': 'luara',
    'luuu': 'luara',
    'lua': 'luara',
    'luaura': 'luara',
    'luaraaa': 'luara',
    'gih': 'giovanna',
    'dionemario': 'dione',
    'gabyzinha': 'gaby',
    'gabriele': 'gaby'
}

## Main

In [39]:
# Main
# Get file info
file_list = [f for f in os.listdir(folder_path)]

df_dict = {}
for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)
    key = file_name
    df_dict[key] = pd.read_excel(
        file_path, sheet_name='Final Scores', usecols="A:C", header=2
    )

# Create dataframe for podium
main_podium = pd.DataFrame()
# Join the files
for key in df_dict:
    podium = df_dict[key].rename(columns={df_dict[key].columns[0]: 'Podium'})
    main_podium = pd.concat([main_podium, podium])

# Rename columns
main_podium.rename(columns={'Total Score (points)': 'Points'}, inplace=True)
# Change data type
main_podium['Podium'] = main_podium['Podium'].astype(int)
main_podium['Points'] = main_podium['Points'].astype(int)

# Clean names
main_podium['Player'] = main_podium['Player'].apply(
    lambda x: clean_string(x))

# Substitute name aliases
main_podium['Player'] = main_podium['Player'].replace(name_alias)

# Assign Podium points
point_mapping = {1: 3, 2: 2, 3: 1}
main_podium['Podium_Points'] = main_podium['Podium'].map(point_mapping)

In [40]:
# [DEBUG] Display the unique players to verify
logging.debug(f'Unique Players raw:\n{sorted(main_podium['Player'].unique())}')

2025-07-13 09:45:05 [DEBUG] Unique Players raw:
['allan', 'ana', 'ange', 'bidomar', 'brenda', 'da', 'dbv', 'dione', 'evelyn', 'feriza', 'fernando', 'fernandoduarte', 'frank', 'freitas', 'gabriel', 'gaby', 'gustavoe', 'huilike', 'lari', 'lauralice', 'le', 'leo', 'leozinho', 'luara', 'lucas', 'machado', 'maio', 'mardabi', 'mario', 'mateus', 'matheuslopes', 'moisesgago', 'nat', 'natal', 'natalmatheus', 'nath', 'pands', 'pmd', 'raquel', 'rws', 'tefi', 'tobias', 'trentin', 'veruska', 'xandao', 'yasmin']


## Final ranking

In [41]:
# Create final ranking
rank = (main_podium.loc[:, ['Player', 'Podium_Points', 'Points']]
        .groupby(['Player'])
        .sum()
        .reset_index())
# Index starts at 1
rank.index = rank.index + 1

In [42]:
# [DEBUG] Display unique players
logging.debug(f'Unique Players:\n{sorted(main_podium['Player'].unique())}')

2025-07-13 09:45:05 [DEBUG] Unique Players:
['allan', 'ana', 'ange', 'bidomar', 'brenda', 'da', 'dbv', 'dione', 'evelyn', 'feriza', 'fernando', 'fernandoduarte', 'frank', 'freitas', 'gabriel', 'gaby', 'gustavoe', 'huilike', 'lari', 'lauralice', 'le', 'leo', 'leozinho', 'luara', 'lucas', 'machado', 'maio', 'mardabi', 'mario', 'mateus', 'matheuslopes', 'moisesgago', 'nat', 'natal', 'natalmatheus', 'nath', 'pands', 'pmd', 'raquel', 'rws', 'tefi', 'tobias', 'trentin', 'veruska', 'xandao', 'yasmin']


## Podium Variations

In [52]:
logging.info(f'\n[{folder_path}] Ranking por Podium Points e desempate por Points')

rank.sort_values(['Podium_Points', 'Points'],ascending=[False, False]) \
    .head(5) \
    .reset_index(drop=True)

2025-07-13 09:47:37 [INFO] 
[2025-2_ADORACAO] Ranking por Podium Points e desempate por Points


,Player,Podium_Points,Points
0,luara,18.0,58005
1,gaby,13.0,62501
2,ange,12.0,56872
3,natal,5.0,29573
4,veruska,4.0,46449


In [50]:
# TOP 5 PODIUM PTS AND THEN BY ACCUMULATED PTS
logging.info(f'\n[{folder_path}] Ranking por top 5 Accumulated Points e ordenado por Podium Points')

rank.sort_values(['Points'], ascending=False) \
    .head(5) \
    .sort_values(['Podium_Points'], ascending=False) \
    .reset_index(drop=True)

2025-07-13 09:46:55 [INFO] 
[2025-2_ADORACAO] Ranking por top 5 Accumulated Points e ordenado por Podium Points


,Player,Podium_Points,Points
0,luara,18.0,58005
1,gaby,13.0,62501
2,ange,12.0,56872
3,veruska,4.0,46449
4,brenda,2.0,58137


In [53]:
logging.info(f'[{folder_path}] Ranking por Points')

rank.sort_values(['Points'], ascending=False) \
    .head(5) \
    .reset_index(drop=True)

2025-07-13 09:48:02 [INFO] [2025-2_ADORACAO] Ranking por Points


,Player,Podium_Points,Points
0,gaby,13.0,62501
1,brenda,2.0,58137
2,luara,18.0,58005
3,ange,12.0,56872
4,veruska,4.0,46449


In [46]:
file_list

['ADORAÇÃO [01] Relação Rompida.xlsx',
 'ADORAÇÃO [02] O início do altar.xlsx',
 'ADORAÇÃO [03] Altares para recordar.xlsx',
 'ADORAÇÃO [04] Um lugar para mim.xlsx',
 'ADORAÇÃO [05] Manhã e tarde.xlsx',
 'ADORAÇÃO [06] O altar é restaurado.xlsx',
 'ADORAÇÃO [07] Verdadeiros Adoradores.xlsx',
 'ADORAÇÃO [08] A vida de Jesus no altar - 1.xlsx',
 'ADORAÇÃO [09] A vida de Jesus no altar - 2.xlsx',
 'ADORAÇÃO [10] O altar e a palavra.xlsx',
 'ADORAÇÃO [11] O altar e o coração.xlsx',
 'ADORAÇÃO [12] O altar e as três mensagens.xlsx',
 'ADORAÇÃO [13] Chamado ao altar.xlsx']

In [47]:
# [DEBUG] Assiduidade | Presença
main_podium.groupby('Player')['Points'] \
           .count() \
           .sort_values(ascending=False) \
           .reset_index() \
           .rename(columns={'Points': 'Presença'})

,Player,Presença
0,brenda,12
1,ange,12
2,gaby,11
3,natalmatheus,10
4,trentin,10
5,veruska,10
6,luara,9
7,feriza,8
8,fernandoduarte,7
9,evelyn,7
